In [1]:
import os
import numpy as np
import tensorflow as tf
from keras.layers import Dense, Dropout
from keras.models import Sequential
from keras_tuner.tuners import RandomSearch
from keras_tuner.engine.hyperparameters import HyperParameters

In [2]:
train_x = np.load("train_data_X_adasyn.npy")
train_y = np.load("train_data_y_adasyn.npy")
test_x = np.load("test_data_X_adasyn.npy")
test_y = np.load("test_data_y_adasyn.npy")

In [3]:
def get_model(hp):
    model = Sequential()
    model.add(tf.keras.Input(shape=train_x.shape[1:]))
    model.add(Dense( hp.Choice('input', [8, 16, 32]), activation="relu"))
    
    for i in range(hp.Int("n_layers", 2, 10, 2)):
        model.add(Dense(hp.Choice(f"dense{i}", [8, 16, 32, 64, 128]), activation="relu"))
        model.add(Dropout(hp.Float(f"dropout{i}", min_value=0.0, max_value=0.5, step=0.1)))

    model.add(Dense(train_y.shape[1], activation = "softmax"))

    model.compile(loss=tf.keras.losses.CategoricalCrossentropy(from_logits=False),
              optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
              metrics=["acc"])
              
    return model

In [4]:
import time

LOG_DIR = f"{int(time.time())}_ADASYN"

In [5]:
tuner = RandomSearch(
    get_model,
    objective = "val_acc",
    max_trials = 10000000,
    executions_per_trial = 1,
    directory = LOG_DIR
)

In [ ]:
tuner.search(
    x = train_x,
    y = train_y,
    epochs = 5,
    batch_size = 64,
    validation_data = (test_x,test_y))

Trial 11835 Complete [00h 00m 12s]
val_acc: 0.4543825387954712

Best val_acc So Far: 0.4756650924682617
Total elapsed time: 13h 14m 05s

Search: Running Trial #11836

Hyperparameter    |Value             |Best Value So Far 
input             |8                 |32                
n_layers          |2                 |2                 
dense0            |8                 |128               
dropout0          |0.4               |0.1               
dense1            |8                 |128               
dropout1          |0.3               |0.2               
dense2            |32                |8                 
dropout2          |0.1               |0.1               
dense3            |64                |128               
dropout3          |0.3               |0.4               
dense4            |32                |64                
dropout4          |0.2               |0.4               
dense5            |8                 |16                
dropout5          |0.1             